<div class='alert alert-info'>

# Diageo Monthly Reports

</div>

<div class='alert alert-warning'>
    
   <h2>Table of Content</h2>
   <ol>
       <li><a href='#imports'>Imports</a></li>
       <li><a href='#brand'>Brand Selection</a></li>
       <li><a href='#functions'>Functions</a></li>
       <li><a href='#fb-ig'>Facebook and Instagram</a></li>
       <li><a href='#all'>Facebook, Instagram, Twitter and Search</a></li>
           <ul>
               <li><a href='#all-fb-ig'>Facebook and Instagram</a></li>
               <li><a href='#twitter'>Twitter</a></li>
               <li><a href='#search'>Search</a></li>
           </ul>
       <li><a href='#export'>Export</a></li>
   </ol>

</div>

<a id='imports'> </a>

## Imports

> Importing all packages required to successfully run this notebook.

In [1]:
import pandas as pd
import os
import glob
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from functools import reduce
import re

<a id='brand'></a>

## Brand Selection

> Selecting which brand you want the report for.

In [2]:
# Changing current working directory into the the directory where the raw data is stored
if 'notebook' in os.getcwd():
    os.chdir('../data/input')
else:
    os.chdir('../input')

In [3]:
# Dictionary of available brands
brands = {1:'Tanqueray',2:'Captain Morgan',3:'Ciroc',4:'Johnnie Walker',5:'Johnnie Walker - Trevor Noah',6:'Smirnoff',7:'The Singleton',8:'Haig',9:'JB'}

In [4]:
# Endless loop running until a valid option is chosen from the display shown
valid = 0
while not valid:
    try:
        brand = int(input("\n".join("{}: {}".format(k, v) for k, v in brands.items())))
        if (brand > 0) and (brand < len(brands)+1):
            valid += 1
            brand = brands[brand]
    except:
        print('INVALID INPUT!!!! TRY AGAIN!')
        
print(f'You have chosen \x1b[1;34m{brand}\x1b[0m')

1: Tanqueray
2: Captain Morgan
3: Ciroc
4: Johnnie Walker
5: Johnnie Walker - Trevor Noah
6: Smirnoff
7: The Singleton
8: Haig
9: JB1
You have chosen Tanqueray


In [5]:
# Reading all the file names in the current working directory that have the extension '.csv' and reading the  first element to a DataFrame
fb_ig = pd.read_csv(glob.glob('*.csv')[0], index_col='Platform').fillna('-')

# Renaming the missing index value with 'total'
fb_ig = fb_ig.reset_index().fillna('total')
fb_ig.set_index('Platform',inplace=True)

# Sorting index values in ascending order
fb_ig.sort_index(inplace=True)

In [6]:
# Changing all column names into lower case
fb_ig.columns = map(str.lower, fb_ig.columns)

# Renaming a few column names into their prefered names
fb_ig.rename(columns={'thruplays':'views','cost per thruplay':'cpv','amount spent (zar)':'spend',
                      'link clicks':'clicks','cost per post engagement':'cpe','cost per lead':'cpl'}, inplace=True)

In [7]:
fb_ig

,reach,"cost per 1,000 people reached",impressions,"cpm (cost per 1,000 impressions)",clicks,cpc (cost per link click),ctr (link click-through rate),frequency,views,cpv,post engagement,cpe,leads,cpl,spend,reporting starts,reporting ends
Platform,,,,,,,,,,,,,,,,,
facebook,2896898,51.713376,11318326,13.235913,1,149808,8.83523e-06,3.907050,13600,11.015322,154139,0.971904,-,-,149808.376293,2020-02-01,2020-02-29
instagram,941057,72.256700,3029670,22.443921,-,-,-,3.219433,12251,5.550377,85073,0.799286,-,-,67997.673707,2020-02-01,2020-02-29
total,3372546,64.582084,14347996,15.180242,1,217806,6.96961e-06,4.254351,25851,8.425440,239212,0.910515,-,-,217806.050000,2020-02-01,2020-02-29


<a id='functions'></a>

## Functions

> Functions used throughout the notebook.

In [8]:
def merge_df(mdf,df):
    '''Merging the two datasets passed into the function. Retaining the index and filling all null values with"-".'''
    
    df = df.append([mdf]).fillna('-')
    if 'search' in df.index:
        df = df.reindex(index=['facebook','instagram','twitter','search','total'])
    else:
        df = df.reindex(index=['facebook','instagram','twitter','total'])
    return(df)

In [9]:
def decimalPlace(ndf):
    '''Rounding all float type values to 2 decimal place.'''
    
    for col in ndf.columns:
        ndf[col] = [round(ndf[col].iloc[i] ,2) if (type(ndf[col].iloc[i]) == np.float64) or 
                    (type(ndf[col].iloc[i]) == float) else ndf[col].iloc[i] for i in range(len(ndf))]
    return(ndf)

<a id='fb-ig'></a>

## Facebook and Instagram

In [10]:
def fb_df(odf,ndf):
    '''Slicing the data features into the required data features (metrics) for the chosen brand. For Facebook and 
    Instagram only'''
    
    if (brand == 'Tanqueray') or (brand == 'Captain Morgan'):
        cols = map(str.lower,['Reach','Impressions','CPM','Frequency','Views','CPV','Post engagement',
                              'CPE','Leads','CPL','Spend'])
        for col in cols:
            dfc = [i for i in odf.columns if col in i]
            if dfc:
                ndf[col] = odf[dfc[0]]
        return(ndf)
    elif (brand == 'Smirnoff') or (brand == 'The Singleton') or (brand == 'JB'):
        cols = map(str.lower,['Reach','Impressions','CPM','Frequency','Views','CPV','Post engagement','CPE','Spend'])
        for col in cols:
            dfc = [i for i in odf.columns if col in i]
            if dfc:
                ndf[col] = odf[dfc[0]]
        return(ndf)
    else:
        cols = map(str.lower,['Reach','Impressions','CPM','Frequency','Post engagement','CPE','Spend'])
        for col in cols:
            dfc = [i for i in odf.columns if col in i]
            if dfc:
                ndf[col] = odf[dfc[0]]
        if brand == 'Ciroc':
            ndf.drop('total', inplace=True)
        return(ndf)

In [11]:
# Creating a new DataFrame for Facebook and Instagram by calling the appropriate function
fb_ig1 = pd.DataFrame()
fb_ig1 = fb_df(fb_ig,fb_ig1)

In [12]:
fb_ig1

,reach,impressions,cpm,frequency,views,cpv,post engagement,cpe,leads,cpl,spend
Platform,,,,,,,,,,,
facebook,2896898,11318326,13.235913,3.907050,13600,11.015322,154139,0.971904,-,-,149808.376293
instagram,941057,3029670,22.443921,3.219433,12251,5.550377,85073,0.799286,-,-,67997.673707
total,3372546,14347996,15.180242,4.254351,25851,8.425440,239212,0.910515,-,-,217806.050000


In [13]:
# Rounding off all float data types to 2 decimal places
fb_ig1 = decimalPlace(fb_ig1)

In [14]:
# Converting columns into index and vice versa
fb_ig1.transpose()

Platform,facebook,instagram,total
reach,2896898,941057,3372546
impressions,11318326,3029670,14347996
cpm,13.24,22.44,15.18
frequency,3.91,3.22,4.25
views,13600,12251,25851
cpv,11.02,5.55,8.43
post engagement,154139,85073,239212
cpe,0.97,0.8,0.91
leads,-,-,-
cpl,-,-,-


os.chdir('../output')
fb_ig1.transpose().to_excel(f'{brand}.xlsx', sheet_name='FB-IG')
print(f'Successfully exported data to \x1b[7;32m{brand}.xlsx\x1b[0m')

<a id='all'></a>

## Facebook, Instagram, Twitter, and Search

<a id='all-fb-ig'></a>

### Facebook and Instagram

In [15]:
def all_fb_df(odf,ndf):
    '''Slicing the data features into the required data features (metrics) for the chosen brand. For Facebook and Instagram 
    for all platforms.'''
    
    if (brand == 'Tanqueray'):
        cols = map(str.lower,['Reach','Impressions','CPM','Clicks','CPC','CTR','Frequency','Views','CPV',
                              'Post engagement','CPE','Leads','CPL','Spend'])
        for col in cols:
            dfc = [i for i in odf.columns if col in i]
            if dfc:
                ndf[col] = odf[dfc[0]]
        return(ndf)
    elif (brand == 'Captain Morgan'):
        cols = map(str.lower,['Reach','Impressions','CPM','Frequency','Views','CPV','Post engagement','CPE',
                              'Leads','CPL','Spend'])
        for col in cols:
            dfc = [i for i in odf.columns if col in i]
            if dfc:
                ndf[col] = odf[dfc[0]]
        return(ndf)
    elif (brand == 'Johnnie Walker') or (brand == 'Johnnie Walker - Trevor Noah'):
        cols = map(str.lower,['Reach','Impressions','CPM','Clicks','CPC','CTR','Frequency','Views','CPV',
                              'Post engagement','CPE','Spend'])
        for col in cols:
            dfc = [i for i in odf.columns if col in i]
            if dfc:
                ndf[col] = odf[dfc[0]]
        return(ndf)
    else:
        cols = map(str.lower,['Reach','Impressions','CPM','Frequency','Views','CPV','Post engagement','CPE','Spend'])
        for col in cols:
            dfc = [i for i in odf.columns if col in i]
            if dfc:
                ndf[col] = odf[dfc[0]]
        return(ndf)

In [16]:
# Creating a new DataFrame for all platforms starting with Facebook and Instagram by calling the appropriate function
all_df = pd.DataFrame()
all_df = all_fb_df(fb_ig,all_df)

In [17]:
all_df

,reach,impressions,cpm,clicks,cpc,ctr,frequency,views,cpv,post engagement,cpe,leads,cpl,spend
Platform,,,,,,,,,,,,,,
facebook,2896898,11318326,13.235913,1,149808,8.83523e-06,3.907050,13600,11.015322,154139,0.971904,-,-,149808.376293
instagram,941057,3029670,22.443921,-,-,-,3.219433,12251,5.550377,85073,0.799286,-,-,67997.673707
total,3372546,14347996,15.180242,1,217806,6.96961e-06,4.254351,25851,8.425440,239212,0.910515,-,-,217806.050000


<a id='twitter'></a>

### Twitter

In [18]:
def twit(df):
    '''Slicing the data features into the required data features (metrics) for the chosen brand. For Twitter for
    all platforms.'''
    
    twitter = pd.read_excel([f for f in glob.glob("*.xlsx") if "2020" in f ][0])
    twitter = twitter[twitter.Impressions > 0]

    twitter.columns = map(str.lower, twitter.columns)
    twitter.rename(columns={'cost per 1k impressions':'cpm','video views':'views','cost per video view':'cpv',
                            'tweet engagements':'post engagement','cost per engagement':'cpe'}, inplace=True)

    twitter = twitter[['impressions','cpm','views','cpv','post engagement','cpe','spend']]

    global new_twitter
    new_twitter = pd.DataFrame()
    new_twitter = twitter_df(twitter, new_twitter)
    new_twitter = decimalPlace(new_twitter)
    
    df = merge_df(new_twitter,df)
    
    df = all_twit_df(df.transpose())
    
    return(df)

In [19]:
def twitter_df(odf,ndf):
    '''Slicing the data features into the required data features (metrics) for the chosen brand. For Twitter.'''
    
    twit_tot = list(map(str.lower,['Impressions','Views','Post engagement','Spend']))
    for col in odf.columns:
        if col in twit_tot:
            ndf[col] = [reduce(lambda a,b:a+b,odf[col])]
        else:
            ndf[col] = [0]

    ndf['cpm'] = [ndf['spend'].iloc[0]/ndf['impressions'].iloc[0]*1000]
    if ndf['views'].iloc[0] > 0:
        ndf['cpv'] = [ndf['spend'].iloc[0]/ndf['views'].iloc[0]]
    else:
        ndf['views'].iloc[0] = None
        ndf['cpv'].iloc[0] = None
    ndf['cpe'] = [ndf['spend'].iloc[0]/ndf['post engagement'].iloc[0]]
    ndf.rename(index={0:'twitter'},inplace=True)
    ndf.dropna(axis=1, inplace=True)
    return(ndf)

In [20]:
def all_twit_df(df):
    '''Calculating the total values for the current total values including Twitter.'''
    
    # Spend total
    df['total'].loc['spend'] = df['total'].loc['spend'] + df['twitter'].loc['spend']
    
    # Views total + CPV total
    if df['twitter'].loc['views'] != '-':
        df['total'].loc['views'] = df['total'].loc['views'] + df['twitter'].loc['views']
        df['total'].loc['cpv'] = df['total'].loc['spend'] / df['total'].loc['views']

    # Post engagement total
    df['total'].loc['post engagement'] = df['total'].loc['post engagement'] + df['twitter'].loc['post engagement']

    # Impressions total
    df['total'].loc['impressions'] = df['total'].loc['impressions'] + df['twitter'].loc['impressions']

    # CPM total
    df['total'].loc['cpm'] = df['total'].loc['spend'] / df['total'].loc['impressions'] * 1000

    # CPL total
    if ((brand == 'Tanqueray') or (brand == 'Captain Morgan')) and (df['total'].loc['leads'] != '-'):
        df['total'].loc['cpl'] = df['total'].loc['spend'] / df['total'].loc['leads']


    # CPE total
    df['total'].loc['cpe'] = df['total'].loc['spend'] / df['total'].loc['post engagement']
    
    return(df.transpose())

In [21]:
# Creating a new DataFrame for twitter by calling the appropriate function provided the selected brand is neither Ciroc nor haig
if (brand == 'Ciroc') or (brand == 'Haig'):
    pass
else:
    all_df = twit(all_df)

In [22]:
# Converting columns into index and vice versa
all_df.transpose()

,facebook,instagram,twitter,total
reach,2.8969e+06,941057,-,3.37255e+06
impressions,11318326,3029670,1645957,15993953
cpm,13.2359,22.4439,54.61,19.2375
clicks,1,-,-,1
cpc,149808,-,-,217806
ctr,8.83523e-06,-,-,6.96961e-06
frequency,3.90705,3.21943,-,4.25435
views,13600,12251,163495,189346
cpv,11.0153,5.55038,0.55,1.62498
post engagement,154139,85073,71676,310888


<a id='search'></a>

### Search

In [23]:
def Search(df):
    '''Slicing the data features into the required data features (metrics) for the chosen brand. For Search for
    all platforms.'''
    
    search = pd.read_excel([f for f in glob.glob("*.xlsx") if "Campaign" in f ][0])
    
    match = 0
    for col in search.columns:
            if re.search(('Unnamed'), col, re.IGNORECASE):
                match+=1
    if match:
        col_name_indx = [i for i in range(len(search)) if search.iloc[i].isnull().sum() == 0][0]
        search.columns = search.rename(index={1:''}).iloc[1]
        search.drop([0,col_name_indx], axis=0, inplace=True)
        search.rename(index={1:''})
        search.reset_index(inplace=True)
        search.drop('index', axis=1,inplace=True)

    search.rename(columns={'Impr.':'Impressions','Cost':'Spend','Avg. CPC':'CPC','Search impr. share':'Impression Share'}, inplace=True)
    search.columns = map(str.lower, search.columns)
    
    global new_search
    new_search = pd.DataFrame([0])
    new_search = search_df(search, new_search)
    new_search = decimalPlace(new_search)
    
    df = merge_df(new_search.drop(columns='impression share'),df)
    
    df = all_search_df(df.transpose())
    
    return(df)

In [24]:
def search_df(odf,ndf):
    '''Slicing the data features into the required data features (metrics) for the chosen brand. For Search.'''
    
    search_tot = list(map(str.lower,['Impressions','Impression Share','CPM','Clicks','CPC','CTR','Spend']))
    
    for col in search_tot:
        if col in odf.columns:
            ndf[col] = odf[col].iloc[-1]
        else:
            ndf[col] = [0]
    ndf.drop(0, axis=1, inplace=True)
    ndf['cpm'] = [ndf['spend'].iloc[0]/ndf['impressions'].iloc[0]*1000]
    ndf['ctr'] = ndf['ctr'].iloc[0]*100
    ndf['impression share'] = ndf['impression share']*100
    ndf.rename(index={0:'search'},inplace=True)
    ndf.dropna(axis=1, inplace=True)
    return(ndf)

In [25]:
def all_search_df(df):
    '''Calculating the total values for the current total values including Search.'''
    
    # Spend total
    df['total'].loc['spend'] = df['total'].loc['spend'] + df['search'].loc['spend']

    # Clicks total
    if df['total'].loc['clicks'] != '-':
        df['total'].loc['clicks'] = df['total'].loc['clicks'] + df['search'].loc['clicks']
    
    # Impressions total
    df['total'].loc['impressions'] = df['total'].loc['impressions'] + df['search'].loc['impressions']

    # CPM total
    df['total'].loc['cpm'] = df['total'].loc['spend'] / df['total'].loc['impressions'] * 1000

    # CPC total
    df['total'].loc['cpc'] = df['total'].loc['spend'] / df['total'].loc['clicks']

    # CTR total
    df['total'].loc['ctr'] = df['total'].loc['clicks'] / df['total'].loc['impressions'] * 100

    # CPL total
    if df['total'].loc['leads'] != '-':
        df['total'].loc['cpl'] = df['total'].loc['spend'] / df['total'].loc['leads'] 
    
    return(df.transpose())

In [26]:
# Creating a new DataFrame for Search by calling the appropriate function provided the selected brand is Tanqueray
if brand == 'Tanqueray':
    all_df = Search(all_df)

In [27]:
# Rounding off all float data types to 2 decimal places
all_df = decimalPlace(all_df)

In [28]:
# Converting columns into index and vice versa
all_df.transpose()

,facebook,instagram,twitter,search,total
reach,2.8969e+06,941057,-,-,3.37255e+06
impressions,11318326,3029670,1645957,14957,16008910
cpm,13.24,22.44,54.61,532.99,19.72
clicks,1,-,-,856,857
cpc,149808,-,-,9.31,368.33
ctr,0,-,-,5.72,0.01
frequency,3.91,3.22,-,-,4.25
views,13600,12251,163495,-,189346
cpv,11.02,5.55,0.55,-,1.62
post engagement,154139,85073,71676,-,310888


<a id='export'></a>

### Export

In [29]:
# Changing the current working directory (input) to the data folder "output" for the processed data
os.chdir('../output')

# Brands Ciroc and Haig, the only brands with only one platform (Facebook)
if (brand == 'Ciroc') or (brand == 'Haig'):
    with pd.ExcelWriter(f'{brand}.xlsx') as writer:
        fb_ig1.transpose().to_excel(writer, sheet_name='FB-IG')
        
# Brand Tanqueray,the only brand with only four platforms (Facebook, Instagram, Twitter and Search)
elif (brand == 'Tanqueray'):
    with pd.ExcelWriter(f'{brand}.xlsx') as writer:  
        all_df.transpose().to_excel(writer, sheet_name='All')
        fb_ig1.transpose().to_excel(writer, sheet_name='FB-IG')
        new_twitter.transpose().to_excel(writer, sheet_name='Twitter')
        new_search.transpose().to_excel(writer, sheet_name='Search')
        
# The rest of the brands, as they have three platforms (Facebook, Instagram and Twitter)
else:
    with pd.ExcelWriter(f'{brand}.xlsx') as writer:  
        all_df.transpose().to_excel(writer, sheet_name='All')
        fb_ig1.transpose().to_excel(writer, sheet_name='FB-IG')
        new_twitter.transpose().to_excel(writer, sheet_name='Twitter')
print(f'Successfully exported data to \x1b[7;32m{brand}.xlsx\x1b[0m')

Successfully exported data to Tanqueray.xlsx
